<a href="https://colab.research.google.com/github/venusinfire/SenalesySistemas2023-2/blob/main/3_SerieyTransformadaFourier/Copia_de_Copia_de_Sounds_basic_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clasificador de canciones

#### Importar las librerias necesarias

In [ ]:
try:
    import yt_dlp as youtube_dl
except:
    !python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

from IPython.display import Audio
import matplotlib.pyplot as plt
import yt_dlp as youtube_dl
from urllib import request
import subprocess as sp
import soundfile as sf
import pandas as pd
import numpy as np
import json
import os
f = os.system('cls' if os.name == 'nt' else 'clear') #eliminar resultados de ejecucion

https://drive.google.com/file/d/10SIJAlmNV2ec1_WbKv_R2alqUMSDVaoE/view?usp=share_link

In [ ]:
FILEID = "1GhRQ14-fRahE9tJmekQ9ggn5zXJORGRCL2SfJqq3Qwg"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O Datos1.zip && rm -rf /tmp/cookies.txt
!unzip Datos1.zip
!dir

--2023-10-19 22:33:05--  https://docs.google.com/uc?export=download&confirm=&id=1GhRQ14-fRahE9tJmekQ9ggn5zXJORGRCL2SfJqq3Qwg
Resolving docs.google.com (docs.google.com)... 172.217.219.139, 172.217.219.102, 172.217.219.100, ...
Connecting to docs.google.com (docs.google.com)|172.217.219.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lgekjnbc8n934qcet081pr0hqjinji68/1697754750000/06689292972694442233/*/1GhRQ14-fRahE9tJmekQ9ggn5zXJORGRCL2SfJqq3Qwg?e=download&uuid=9e666ce5-8303-4176-abd0-4f523e7fa448 [following]
--2023-10-19 22:33:05--  https://doc-0g-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lgekjnbc8n934qcet081pr0hqjinji68/1697754750000/06689292972694442233/*/1GhRQ14-fRahE9tJmekQ9ggn5zXJORGRCL2SfJqq3Qwg?e=download&uuid=9e666ce5-8303-4176-abd0-4f523e7fa448
Resolving doc-0g-0g-docs.googleusercontent.com (doc-0g-0g-docs.google

#### Función para descargar los audios

In [ ]:
def download_ytvid_as_mp3(video_url, path):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{path}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    inputData = f'{path}.mp3'
    outputData = f'{path}.wav'
    if not os.path.exists(inputData): #verifica que el audio se descargo correctamente
        print(f"El archivo {inputData} no existe.")
        return
    else: #convertir el auido descargado a formato .wav
        sp.call(['ffmpeg', '-i', inputData, outputData])
        os.remove(inputData) #eliminar el audion en formato mp3
    print(f"Download complete... {filename}")


#### Obtener informacion del video desde la API de youtube

In [ ]:
def get_title(url,label):
    url_Api = 'https://www.youtube.com/oembed?url'
    data = request.urlopen(f"{url_Api}={url}")
    data = json.loads(data.read())
    return data[label]

### Segmentos de interes por canción
se elige un segmentos de de 20 segundos por cancion


In [ ]:
def audio_parts(signal,fs):
    log_t =len(signal)
    med_s = [int((log_t/4)-0*fs),int((log_t/4)+20*fs)]
    return(med_s)

### Función para graficar una señal

In [ ]:
def plt_signal(vf,Xw):
    plt.plot(vf,abs(Xw),linewidth=0.5)#se grafica la magnitud
    plt.legend(('canal 1','canal 2'))
    #plt.title(f'parte: {round(ti,2)} - {round(tf,2)}')
    plt.xlabel(r'$f[Hz]$',fontsize = 14)
    plt.ylabel(r'$|X[n]|$',fontsize = 14)
    plt.show()

def subplot_signal(x1,y1,x2,y2):
    plt.figure(figsize=(12,6))

    plt.subplot(121) # 1 = #filas, 2 = #columnas, 1 = # dibujo
    plt.plot(x1,abs(y1),linewidth=0.5)
    plt.xlabel('$f[Hz]$')
    plt.ylabel('$Ampl$')
    #plt.title(f'parte: {round(ti,2)} - {round(tf,2)}')
    plt.legend(('canal 1','canal 2'))

    plt.subplot(122)
    plt.plot(x2,y2,linewidth=0.5)
    plt.xlabel('t[s]')
    plt.ylabel('$Ampl$')
    #plt.title(f'parte: {round(ti,2)} - {round(tf,2)}')
    plt.legend(('canal 1','canal 2'))
    plt.subplots_adjust(hspace=0.2,wspace=0.2)
    plt.show()

### Procesar Transformada de Fourier a cada parte de las canciones
la sigunete funcion aplica fft a cada secmento tomado por cacion, luego este es guardado en un achivo de extencion .npy

In [ ]:
def fft_song(song,path):
    xpro = song.copy()
    Xw = np.fft.rfft(xpro,axis=0) # axis=0 permite aplicar fft por cada columna de xpro
    np.save(f'{path}',Xw)

### Importar transformadas de Fourier de cada audio
Luego de guardar en memoria externa cada una de las trasformadas, se cargan al prgrama para su respectivo análisis

In [ ]:
def loads_fft(path):
    print(f'leyendo {path}')
    data = np.load(f'{path}.npy')
    return data

### Se crea la clase canción
la clase define todos los atributos y metodos disponibles para un audio

In [ ]:
class Song():
    def __init__(self,link,genero,index):
        self.name = f'{index}_{genero}'
        self.genero = genero
        self.link = link
        self.path_Org = f'{genero}/Org/{self.name}' #ruta de audio original
        self.path_FFT = f'{genero}/FFT/{self.name}' #ruta del audio procesado
        self.vf = [] #vector de frecuencia
        self.sections = [] #amacena el intervalo de tiempo de los segmentos tomados del audio
        self.fft_sections = [] #matriz: almacena la transformada de cada segmento

    def download(self):     #descargar audio y guardar en formato.wav
        download_ytvid_as_mp3(self.link,self.path_Org)

    def load_song(self):    #importar la informacion desde un archivo en memoria
        #self.read_x()
        try:
            self.fft_sections = loads_fft(f'{self.path_FFT}') #leer archivo que contiene las trasforamas de la cancion
        except:
            print(f'{self.path_FFT}: Error en la lectrura de los datos. intente descargar el audio')

        #self.vf = np.fft.rfftfreq(int(16*self.fs),1/self.fs) #se crea el vector de frecuencias

    def read_x(self):
        try:
            x_T, self.fs = sf.read(f'{self.path_Org}.wav') #leer la cancion original
        except:
            self.download()
            x_T, self.fs = sf.read(f'{self.path_Org}.wav')
        self.sections = audio_parts(x_T,self.fs) #dividir el archivo en secciones
        x = x_T[self.sections[0]:self.sections[1],:]
        return x

    def process_song(self,plot=False):
        try:
            os.makedirs(os.path.join(self.genero,'FFT')) #se cra una carpeta pra almacenar la transformada correspondiente
        except:
            pass
        x = self.read_x()
        fft_song(x,self.path_FFT) #aplicar transformada a cada seccion y guardar

    def plot_sections(self,dual=False):
        try:
            print("Descargando datos")
            self.vf = np.fft.rfftfreq(int(20*self.fs),1/self.fs) #se crea el vector de frecuencias
        except:
            self.download()
            x_T, self.fs = sf.read(f'{self.path_Org}.wav')
            self.vf = np.fft.rfftfreq(int(20*self.fs),1/self.fs) #se crea el vector de frecuencias


        if dual == True:
            x = self.read_x()
            tt = np.arange(0,20,1/self.fs)
            subplot_signal(self.vf,self.fft_sections,tt,x)
        else:
            plt_signal(self.vf,self.fft_sections)

    def play_section(self):
        x = self.read_x()
        return x.T

    def title(self):
        return get_title(self.link,'title')

### Funcion para extraer link y nombres desde una base de datos de excel


In [ ]:
def links(genero):
    path = 'links.xlsx'
    id = 0
    data = pd.read_excel(path, sheet_name=genero)
    songs = []
    for index, row in data.iterrows():
        id += 1
        #title = get_title(row['Links'],'title')
        #print(f'Titulo {id-1}: {title}')
        songs.append(Song(row['Links'],genero,id))

    print(f'Catidad de datos cargados de {genero}: {id}')
    return songs

### Iniciar la descarga de todos los archivos

Procesar cada audio y guardarlo en una matriz de datos

In [ ]:
generos = ['Metal', 'Rock']
db = [links (i) for i in generos]

FileNotFoundError: ignored

In [ ]:
def load(db):
    i = 0
    data = []
    for genero in db:
        data_genero = []
        for song in genero:
            try:  #si la cancion no esta descargada se descarga primero
                song.load_song()
            except:
                song.download()
                song.process_song()
                song.load_song()
            data_genero.append(song)
            print('-----------------------------------------')
        data.append(data_genero)
    return(data)

data = load(db)

In [ ]:
generod = 1
song = 6
Audio(data[generod][song].play_section(),rate=data[generod][song].fs)

In [ ]:
data[generod][song].plot_sections()

### Estimación del genero
Se define una función que dada una canción compare esta su transformada con cada una de las trasformadas de cada canción de los distintos géneros en la base de datos, una vez terminada la comparación retorna la canción de la base de datos más parecida a la entrada, para esto se evalúa la diferencia de la trasformada de cada comparación y se halla su promedio, la canción con el menor promedio de diferencia corresponde a la canción más similar a la entrada, el género de la canción de entrada corresponde al género de la canción más parecida.

In [ ]:
def get_genero(s_input):
    dp = s_input.fft_sections
    #prom_data = []
    best = 500
    for genero in data:
        prom = []
        i = 0
        for song in genero:
            dis_c1 = np.abs(dp[:][0]-song.fft_sections[:][0])
            dis_c2 = np.abs(dp[:][1]-song.fft_sections[:][1])
            prom.append(np.abs(np.mean(dis_c1)-np.mean(dis_c2)))

            if prom[i] <= 5:
                #print(f'{song.genero} prom: {prom[i]}: {song.link}')
                pass
            i+=1

        if min(prom) < best:
            best = min(prom)
            #g = base[np.argmin(prom)].genero
            g = genero[np.argmin(prom)]

        #prom_data.append(np.mean(prom))

    #print(f'\nGenero: {generos[np.argmin(prom_data)]}')
    return(g)

### Pruebas
Se indroducen canciones y se estima su genero

##Menu de pruebas manuales


In [ ]:
index = 50 #correr solo la primera vez

In [ ]:
link = input('https://www.youtube.com/watch?v=W5b-a8Nzmag: ')
S1 = Song(link,'pruebas',index)
index += 1

S1.download()
S1.process_song()
S1.load_song()

g = get_genero(S1)
print('-'*50)
print(f'El genero de {S1.title()} es: {g.genero}')





In [ ]:
Audio(S1.play_section(),rate=S1.fs)


In [ ]:
#S1.plot_sections(dual=True)
#g.process_song
g.load_song()
#g.path_Org
#g.plot_sections()

In [ ]:
get_genero(S2).genero

Carra: 84.45558357679155
elec: 45.504129447204974
rock: 50.21328614230515
126.268797207874

In [ ]:
test_elec = ['https://www.youtube.com/watch?v=3YZgu0nHImA',
            'https://www.youtube.com/watch?v=BGNyuNqAQPI',
            'https://www.youtube.com/watch?v=W5b-a8Nzmag',
            'https://www.youtube.com/watch?v=5JxgDJvqGmM',
            'https://www.youtube.com/watch?v=zaIsVnmwdqg',
            'https://www.youtube.com/watch?v=uY1QVrQJ16w',
            'https://www.youtube.com/watch?v=vAEwLvxHVVk',
            'https://www.youtube.com/watch?v=rIkPh9d_b1Y',]



test_rock = ['https://www.youtube.com/watch?v=M2cckDmNLMI',
            'https://www.youtube.com/watch?v=zPtyAzve2Dc',
            'https://www.youtube.com/watch?v=scohx9trwrw',
            'https://www.youtube.com/watch?v=PCp2iXA1uLE',
            'https://www.youtube.com/watch?v=RWwrk9PMB28',
            'https://www.youtube.com/watch?v=psUh0fbszsw',
            'https://www.youtube.com/watch?v=KrRcxv7pkvs',
            'https://www.youtube.com/watch?v=LnlgkuAak5Y',
            'https://www.youtube.com/watch?v=ThI5qL-28a0',
            'https://www.youtube.com/watch?v=2uFwEXhBihI']

test_all = [test_elec,test_rock]

In [ ]:
index = 0


Metal = []
Rock = []
genero = 0

for gn in test_all:

    if genero == 1:
        j = 'Rock'
    elif genero == 2:
        j = 'Metal'

    for i in gn:
        S2 = Song(i,f'input_pruebas_{1}',index)
        index +=1
        try:
            S2.process_song()
            S2.load_song()
        except:
            S2.download()
            S2.process_song()
            S2.load_song()
        print('----------------------------------------------------------------------------')

        g = get_genero(S2)
        g = g.genero
        print(S2.title())
        #print(S2.link)
        print(f'Genero: {g}')
        print(f'Genero Real: {j}')

        if g == j:
            res = 'Correcto'
        elif g == 'None':
            res = 'None'
        else:
            res = 'Incorrecto'

        if j == 'Rock':
            Rock.append(res)
        elif j == 'Metal':
            Metal.append(res)

    genero += 1

print(f"Metal: {Metal.count('Correcto')} de {len(Metal)}")
print(f"Rock: {Rock.count('Correcto')} de {len(Rock)}")